# 什么是模型导出

当我们使用深度学习模型进行实际生产过程中的服务时，很多时候不会直接使用原始的模型文件（比如，PyTorch的pytorch_model.bin等需要通过load_state_dict加载的二进制文件），
这是因为这些模型文件只能在python环境中运行，而且无法进行"优化"。举个例子，假如某个环境是嵌入式设备的单片机，那么在它的上面安装python环境是非常费力的；另外如果模型比较大，想要
达到比较高的QPS，模型需要摆脱python环境而使用更快的C++库，或进行一些硬件相关的优化，或进行一些算子的融合。

在这种情况下，使用原始的代码和二进制文件运行就比较得不偿失，因此各类
算法库都提供了对应的"导出格式"，比如PyTorch的TorchScript，tensorflow的GraphDef，或者跨框架格式ONNX。这些格式不仅包含了模型的各类参数，也包含了模型动态图本身，因此可以脱离
python环境独立运行并可以获得一定的运行加速，不少算子库也支持以这些格式为起点进行后续优化。ModelScope提供了其内模型的导出方法，用户可以自由选用。

# 导出为ONNX格式

[ONNX](https://onnx.ai/) 全称为开放神经网络交换格式（Open Neural Network Exchange），是微软和Facebook（Meta）联合提出用于表示深度学习模型的文件格式。
其特点为标准的文件格式，且具备平台无关性。也就是说，用户在任意框架（TensorFlow/PyTorch/JAX等）中训练得到的原始模型
都可以转换为这种格式进行存储和优化，或转换为其他框架专用的模型文件。ONNX文件和其他输出格式一样，不仅存储了模型权重，
也存储了模型DAG图以及一些有用的辅助信息。

如果您的生产环境使用ONNX，或需要ONNX格式进行后续优化，您可以使用ModelScope提供的ONNX转换工具进行模型导出。

## 导出方法

首先我们需要初始化一个已支持Exporter模块的模型：


In [1]:
from modelscope.models import Model
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
model = Model.from_pretrained(model_id)


下面我们就可以将其导出为对应格式：


In [1]:
from modelscope.exporters import Exporter
# shape参数用来生成dummy inputs
# 在NLP领域中一般len(shape) == 2, 分别代表batch_size和sequence_length
output_files = Exporter.from_model(model).export_onnx(shape=(2, 256), opset=13, output_dir='/tmp')
print(output_files) # {'model': '/tmp/model.onnx'}


opset是onnx算子版本，具体可以参考[这里](https://onnxruntime.ai/docs/reference/compatibility.html)。

在导出完成后，ModelScope会使用dummy_inputs验证onnx文件的正确性，因此如果导出过程不报错就证明导出过程已经成功了。

需要注意的是，验证过程需要onnx包和onnxruntime包，如果您的环境中没有安装，会看到如下报错：


In [1]:
"modelscope - WARNING - Cannot validate the exported onnx file, because the installation of onnx or onnxruntime cannot be found"


如果需要验证过程可以安装这两个包：
```shell
pip install onnx
pip install onnxruntime
```
或使用conda命令安装：
```shell
conda install -c conda-forge onnx
conda install -c conda-forge onnxruntime
```

如果需要在GPU环境下进行验证过程，可以改为使用下面的命令：
```shell
pip install onnx
pip install onnxruntime-gpu
```
onnxruntime-gpu和CUDA版本和cuDNN版本强相关，安装时请注意版本对应。具体可以参考[这里](https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html)。

## 如何在外部模型上使用导出功能

如果目前支持导出的模型中没有您需要的模型，或该模型是一个torch.nn.Module，可以手动传入dummy_inputs，inputs和outputs来实现。

如下展示了导出transformers库模型的示例。首先把模型和tokenizer初始化出来：


In [1]:
from transformers import BertForSequenceClassification, BertTokenizerFast
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


之后我们使用tokenizer来生成dummy_inputs并调用导出工具：


In [1]:
from modelscope.exporters import TorchModelExporter
from collections import OrderedDict
# 假设最大支持256长度的句子
dummy_inputs = tokenizer(tokenizer.unk_token, padding='max_length', max_length=256, return_tensors='pt')
dynamic_axis = {0: 'batch', 1: 'sequence'}
inputs = OrderedDict([
    ('input_ids', dynamic_axis),
    ('attention_mask', dynamic_axis),
    ('token_type_ids', dynamic_axis),
])
outputs = OrderedDict({'logits': {0: 'batch'}})
output_files = TorchModelExporter().export_onnx(model=model, dummy_inputs=dummy_inputs, inputs=inputs, 
                                                outputs=outputs, output_dir='/tmp')
print(output_files) # {'model': '/tmp/model.onnx'}


inputs和outputs参数用来指示动态dimension，其格式为OrderedDict，key为输入/输出参数名，
value为格式如{0: 'batch', 1: 'sequence'}的动态dimension序号和名称。例子中的0代表tensor第一个维度，1代表tensor第二个维度，
batch/sequence为自定义的维度名称。

在实现上，ModelScope调用了torch.onnx.export方法用以导出onnx，这个方法的输入实际上需要一个ScriptModule，但是也兼容
torch.nn.Module。如果传入模型不是ScriptModule，export方法内部会使用trace方式将模型转为ScriptModule。由于模型结构的复杂性，
大部分ModelScope模型尚不支持script方式进行模型导出，这一部分我们仍在探索中。
有关trace和script方式的使用可以参考[官方文档](https://pytorch.org/docs/stable/onnx.html#tracing-vs-scripting)。


## 支持导出ONNX的ModelScope模型


| 模型         |                       任务 |
|------------|-------------------------:|
| structbert |                      nli |
| structbert | sentiment-classification |
| structbert |      sentence-similarity |
| structbert | zero-shot-classification |

注意，这里指的支持导出是Exporter中存在对应某一模型的具体实现，用户仍然可以使用上述的外部模型导出手动定制自己的导出过程。


## 如何使用ONNX模型

首先需要安装onnxruntime运行时环境，onnxruntime支持多种语言多个平台，具体可以参考[这里](https://onnxruntime.ai/)。

为简便演示，我们在这里展示了python环境中onnxruntime的使用方法，模型为上面外部模型导出的onnx文件，onnxruntime安装过程可以参考上面的文档。

首先构造inputs：


In [1]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
dummy_inputs = tokenizer('这是一个测试的例子', padding='max_length', max_length=256, return_tensors='np')


调用onnxruntime来运行模型：


In [1]:
import onnxruntime as ort
ort_session = ort.InferenceSession('/tmp/model.onnx')
outputs = ort_session.run(['logits'], dict(dummy_inputs))
print(outputs)



# 导出为TorchScript格式

同ONNX类似，[TorchScript](https://pytorch.org/docs/master/jit.html)也是深度学习模型的中间表示格式，不同的是它是基于PyTorch框架的。
Torch模型通过导出变为TorchScript格式后，就可以脱离python环境运行或进行后续的推理加速。

ModelScope也提供了模型转为TorchScript的能力。

## 导出方法

首先我们需要初始化一个已支持导出TorchScript的模型：


In [1]:
from modelscope.models import Model
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
model = Model.from_pretrained(model_id)


下面我们就可以将其导出为对应格式：


In [1]:
# 由于TorchScript是Pytorch特有的格式，因此需要使用TorchModelExporter
from modelscope.exporters import TorchModelExporter
# shape参数用来生成dummy inputs
# 在NLP领域中一般len(shape) == 2, 分别代表batch_size和sequence_length
output_files = TorchModelExporter.from_model(model).export_torch_script(shape=(2, 256), output_dir='/tmp')
print(output_files) # {'model': '/tmp/model.ts'}


模型转换TorchScript有两种方式，Script和Trace。Script方式对以加载完毕的模型代码进行静态分析，并生成TorchScript文件。
而Trace方式仍然需要一个dummy input用来追溯模型的动态图，用来后续分析生成。

Script方式的优点是，可以将源代码的特性包含进去，比如if分支条件等。但由于使用了AST方式进行代码分析，其对模型的要求也较高，比如
需要模型在输入参数上有类型标注，方法中没有无法追溯的动态类型等。Trace方式要求较低，只需要一个构造好的dummy input既可根据动态图
生成静态图。但trace方式要求输入全部为tensor，且模型逻辑中不包含tensor无参与的if分支条件，也给导出带来了一定限制。

ModelScope模型大部分都支持trace方式，因此我们把trace方式选择为默认的导出方式。

同样地，在导出完成后，ModelScope会使用dummy_inputs验证ts文件的正确性，因此如果导出过程不报错就证明导出过程已经成功了。

注意：Script方式生成的文件不支持动态尺寸输入，也就是说，用于以后生产环境中的输入tensor尺寸必须和dummy inputs相同。如果实际输入尺寸小于
dummy input尺寸，请注意在数据预处理过程中添加padding。


## 如何在外部模型上使用导出功能

如果目前支持导出的模型中没有您需要的模型，或该模型是一个torch.nn.Module，可以手动传入dummy_inputs来实现。

如下展示了导出transformers库模型的示例。首先把模型和tokenizer初始化出来：


In [1]:
from transformers import BertForSequenceClassification, BertTokenizerFast
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


之后我们使用tokenizer来生成dummy_inputs并调用导出工具：


In [1]:
from modelscope.exporters import TorchModelExporter
from collections import OrderedDict
# 假设最大支持256长度的句子
dummy_inputs = tokenizer(tokenizer.unk_token, padding='max_length', max_length=256, return_tensors='pt')
output_files = TorchModelExporter().export_torch_script(model=model, dummy_inputs=dummy_inputs, output_dir='/tmp')
print(output_files) # {'model': '/tmp/model.ts'}




## 支持导出TorchScript的modelscope模型


| 模型         |                       任务 |
|------------|-------------------------:|
| structbert |                      nli |
| structbert | sentiment-classification |
| structbert |      sentence-similarity |
| structbert | zero-shot-classification |

注意，这里指的支持导出是Exporter中存在对应某一模型的具体实现，用户仍然可以使用上述的外部模型导出手动定制自己的导出过程。


## 如何使用TorchScript模型

TorchScript模型支持多种语言环境，有关使用可以参考[这里](https://pytorch.org/tutorials/advanced/cpp_export.html#step-3-loading-your-script-module-in-c)。

为简便演示，我们在这里展示了python环境中TorchScript的使用方法，模型为上面外部模型导出的ts文件。

首先构造inputs：


In [1]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
dummy_inputs = tokenizer('这是一个测试的例子', padding='max_length', max_length=256, return_tensors='pt')


调用torch来运行模型：


In [1]:
import torch
ts_model = torch.jit.load('/tmp/model.ts')
ts_model.eval()
with torch.no_grad():
    outputs = ts_model.forward(**dummy_inputs)
print(outputs)
